In [14]:
import json
import matplotlib
from matplotlib import pyplot as plt
import pickle
import sklearn
import pandas as pd
%matplotlib inline

In [3]:
oresults_path = '../predicted-results/original'
mresults_path = '../predicted-results/mixed5050'
eresults_path = '../predicted-results/everything'

omodels_path = '../models/original'
mmodels_path = '../models/mixed5050'
emodels_path = '../models/everything'

test_sets = ['5050', '2575', 'everything']

mfractions = [0.05, 0.1, 0.2,
              0.3, 0.5, 0.7, 1]
efractions = [0.01, 0.02, 0.03,
             0.05, 0.1, 0.2, 0.3,
             0.5, 0.7, 1]

In [5]:
results = {'5050': {'COF': dict(),
                    'intercept': dict()},
           '2575': {'COF': dict(),
                    'intercept': dict()},
           'everything': {'COF': dict(),
                          'intercept': dict()}}
# First deal with the original models 
for tset in results:
    for target in ['COF', 'intercept']:
        with open('{}/{}.pickle'.format(omodels_path, target), 'rb') as f:
            model = pickle.load(f)
        with open('{}/{}.ptxt'.format(omodels_path, target), 'rb') as f:
            features = pickle.load(f)
        with open('{}/{}_on_{}.json'.format(oresults_path, target, tset), 'r') as f:
            data = json.load(f)
        results[tset][target]['original'] = {
            'model': model,
            'features': features,
            'data': data[target]
        }

# Then deal with the 5050 mixed models 
for tset in results:
    for fraction in mfractions:
        for target in ['COF', 'intercept']:
            with open('{}/{}_{}.pickle'.format(mmodels_path, target, fraction), 'rb') as f:
                model = pickle.load(f)
            with open('{}/{}_{}.ptxt'.format(mmodels_path, target, fraction), 'rb') as f:
                features = pickle.load(f)
            with open('{}/{}_{}_on_{}.json'.format(mresults_path, target, fraction, tset), 'r') as f :
                data = json.load(f)
            results[tset][target]['5050_{}'.format(fraction)] = {
                'model': model,
                'features': features,
                'data': data[target]
            }
# Lastly deal with the everything models
for tset in results:
    for fraction in efractions:
        for target in ['COF', 'intercept']:
            with open('{}/{}_{}.pickle'.format(emodels_path, target, fraction), 'rb') as f:
                model = pickle.load(f)
            with open('{}/{}_{}.ptxt'.format(emodels_path, target, fraction), 'rb') as f:
                features = pickle.load(f)
            with open('{}/{}_{}_on_{}.json'.format(eresults_path, target, fraction, tset), 'r') as f :
                data = json.load(f)
            results[tset][target]['everything_{}'.format(fraction)] = {
                'model': model,
                'features': features,
                'data': data[target]
            }


In [48]:
worsts = {'5050': {'COF': dict(),
                    'intercept': dict()},
           '2575': {'COF': dict(),
                    'intercept': dict()},
           'everything': {'COF': dict(),
                          'intercept': dict()}}
wfrac = 0.1

for tset in results:
    for target in ['COF', 'intercept']:
        for model, data in results[tset][target].items():
            n_tset = len(data['data'])
            n_worst = int(n_tset*wfrac)
            tmp_dict = dict()

            for i in range(n_tset-1):
#                 tg1 = data['data'][str(i)]['tg-1']
#                 frac1 = data['data'][str(i)]['frac-1']
#                 tg2 = data['data'][str(i)]['tg-2']
#                 frac2 = data['data'][str(i)]['frac-2']
#                 tg3 = data['data'][str(i)]['tg-3']
                simulated = data['data'][str(i)]['simulated-{}'.format(target)]
                predicted = data['data'][str(i)]['predicted-{}'.format(target)]
                tmp_dict[str(i)] = data['data'][str(i)]
                tmp_dict[str(i)]['error'] = abs(simulated-predicted)/simulated
            worsts[tset][target][model] = pd.DataFrame(tmp_dict).transpose().sort_values(by='error', ascending=False).head(n_worst)
            

In [56]:
worsts['everything']['COF']['everything_1']


,tg-1,frac-1,tg-2,frac-2,tg-3,predicted-COF,simulated-COF,error
1088,difluoromethyl,0.25,cyano,0.75,cyano,0.107961,0.0737549,0.463779
433,fluorophenyl,0.5,hydroxyl,0.5,isopropyl,0.134811,0.104748,0.286995
35,acetyl,0.5,cyano,0.5,fluorophenyl,0.121835,0.0959823,0.269354
735,acetyl,0.25,carboxyl,0.75,ethylene,0.116452,0.0925456,0.25832
202,cyano,0.5,ethylene,0.5,carboxyl,0.124633,0.0991214,0.257379
...,...,...,...,...,...,...,...,...
125,amino,0.5,phenyl,0.5,phenyl,0.131112,0.145394,0.0982284
845,amino,0.25,nitrophenyl,0.75,perfluoromethyl,0.140753,0.12818,0.0980948
1078,difluoromethyl,0.25,acetyl,0.75,cyano,0.117924,0.13069,0.0976784
497,hydroxyl,0.5,phenyl,0.5,ethylene,0.123398,0.112481,0.0970562


In [57]:
# Pick system with best performances
# May be the one with lowest friction force at 25 nN normal force?
worsts['everything']['COF']['everything_1'].head(10)

,tg-1,frac-1,tg-2,frac-2,tg-3,predicted-COF,simulated-COF,error
1088,difluoromethyl,0.25,cyano,0.75,cyano,0.107961,0.0737549,0.463779
433,fluorophenyl,0.5,hydroxyl,0.5,isopropyl,0.134811,0.104748,0.286995
35,acetyl,0.5,cyano,0.5,fluorophenyl,0.121835,0.0959823,0.269354
735,acetyl,0.25,carboxyl,0.75,ethylene,0.116452,0.0925456,0.25832
202,cyano,0.5,ethylene,0.5,carboxyl,0.124633,0.0991214,0.257379
1936,pyrrole,0.25,hydroxyl,0.75,ethylene,0.131694,0.106593,0.235484
1156,ethylene,0.25,carboxyl,0.75,cyclopropyl,0.133687,0.108925,0.227337
200,cyano,0.5,difluoromethyl,0.5,nitro,0.133964,0.110165,0.216032
1797,phenol,0.25,isopropyl,0.75,carboxyl,0.128228,0.105506,0.215358
96,amino,0.5,hydroxyl,0.5,carboxyl,0.149901,0.123649,0.212315


In [58]:
print(worsts.keys())

dict_keys(['5050', '2575', 'everything'])


In [59]:
print(worsts['everything'].keys())

dict_keys(['COF', 'intercept'])


In [60]:
print(worsts['everything']['COF'].keys())

dict_keys(['original', '5050_0.05', '5050_0.1', '5050_0.2', '5050_0.3', '5050_0.5', '5050_0.7', '5050_1', 'everything_0.01', 'everything_0.02', 'everything_0.03', 'everything_0.05', 'everything_0.1', 'everything_0.2', 'everything_0.3', 'everything_0.5', 'everything_0.7', 'everything_1'])
